Reference: [spacy NLP guide](https://mahadev001.github.io/Mahadev-Upadhyayula/Sentiment%20Analysis%20via%20NLP/Sentiment%20Analysis%20using%20NLP%20with%20Spacy%20and%20%20SVM.html)

In [ ]:
import os
import sys

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.path += ['/content/drive/Shareddrive/filtered_data']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd /content/drive/Shareddrives/cs145project/filtered_data/stocks
%cd ../..
%cd headlines/sentiment_analysis_financial_news

/content/drive/Shareddrives/cs145project/filtered_data/stocks
/content/drive/Shareddrives/cs145project
/content/drive/Shareddrives/cs145project/headlines/sentiment_analysis_financial_news


In [ ]:
path="all-data.csv"
sentiments=pd.read_csv(path,encoding = "ISO-8859-1",header=None)

In [ ]:
sentiments.columns = ['Sentiment','Text']

In [ ]:
sentiments.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [ ]:
sentiments['Sentiment'].isnull().sum()

0

In [ ]:
import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:

nlp = spacy.load('en_core_web_sm')

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
import string
punct = string.punctuation

In [ ]:
def clean_text(text):
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords and word not in punct) # delete stopwords and punctuations from text
    return text

In [ ]:
sentiments[sentiments.columns[1]] = sentiments[sentiments.columns[1]].apply(clean_text)
sentiments.head()

,Sentiment,Text
0,neutral,according gran company plans production russia...
1,neutral,"technopolis plans develop stages area 100,000 ..."
2,negative,international electronic industry company elco...
3,positive,new production plant company increase capacity...
4,positive,according company updated strategy years 2009-...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split


In [ ]:
def clean_text(text):     
    return text.strip().lower()
class transformers(TransformerMixin):
    def transform(self,X_train):
        return [clean_text(text) for text in X_train]
    def fit(self,X, y):
        return self
    def set_params(self,best_params, set=True):
        return {}

In [ ]:
from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English
# nlp = English()
spacy_tokenizer = Tokenizer(nlp.vocab)

In [ ]:
def spacy_tokenizer(sentence):
  doc = nlp(sentence)
  filtered_tokens = [token.lemma_ for token in doc if not token.is_stop]
  return filtered_tokens

In [ ]:
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()

In [ ]:
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)


In [ ]:
X=sentiments[sentiments.columns[1]]
y=sentiments[sentiments.columns[0]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
model = Pipeline([("cleaner", transformers()),('vectorizer', vectorizer),('classifier', classifier)])
model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.transformers object at 0x7f4ec8f03070>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f4ec8ef6e60>)),
                ('classifier', LinearSVC())])

In [ ]:
prediction = model.predict(X_test)

In [ ]:
predicted_train = model.predict(X_train)
print(f"train accuracy: {accuracy_score(y_train, predicted_train)}")

# Predicting the test data categories
predicted_categories = model.predict(X_test)
print(f"test accuracy: {accuracy_score(y_test, predicted_categories)}")

train accuracy: 0.9979360165118679
test accuracy: 0.7381443298969073


In [ ]:
%cd /content/drive/Shareddrives/cs145project/filtered_data/

/content/drive/Shareddrives/cs145project/filtered_data


In [ ]:
path="top-50-analyst-ratings-processed.csv"
headlines=pd.read_csv(path,encoding = "ISO-8859-1")
headlines['title']=headlines['title'].apply(clean_text)
headlines.head()

,title,date,stock
0,"piper sandler maintains overweight on adobe, l...",2020-03-31 06:18:00-04:00,ADBE
1,shares of several technology companies are tra...,2020-03-30 10:23:00-04:00,ADBE
2,"shares of several technology, semiconductor an...",2020-03-27 11:30:00-04:00,ADBE
3,"cramer reveals stock favorites, says intuitive...",2020-03-27 10:10:00-04:00,ADBE
4,shares of several software companies are tradi...,2020-03-26 10:38:00-04:00,ADBE


In [ ]:
input=headlines['title']
cats=model.predict(input)
headlines['sentiment']=cats

from collections import Counter
counts = Counter(headlines['sentiment'])
print("Sentiment Distribution:")
print("    Sentiment\tcount\t% of dataset")
for sentiment, count in counts.items():
  print(f"    {sentiment}\t{count}\t{((count/counts.total())*100):.3f}%")

headlines.head()

Sentiment Distribution:
    Sentiment	count	% of dataset
    negative	7146	6.108%
    positive	21364	18.261%
    neutral	88483	75.631%


,title,date,stock,sentiment
0,"piper sandler maintains overweight on adobe, l...",2020-03-31 06:18:00-04:00,ADBE,negative
1,shares of several technology companies are tra...,2020-03-30 10:23:00-04:00,ADBE,positive
2,"shares of several technology, semiconductor an...",2020-03-27 11:30:00-04:00,ADBE,negative
3,"cramer reveals stock favorites, says intuitive...",2020-03-27 10:10:00-04:00,ADBE,neutral
4,shares of several software companies are tradi...,2020-03-26 10:38:00-04:00,ADBE,neutral


In [ ]:
headlines.to_pickle("spacy_sentiment.pkl")